# 시작 CPU버전


## 사용법
- ctrl + enter : 셀실행
- 하나의 칸을 셀이라고 말합니다
- 셀을 실행하면 블럭이 쌓이는 느낌
- 셀은 이전 것이 진행되야 다음이 됩니다.
- 메뉴 ->  kernel -> restart를 하면 초기화
- 쓰레기가 많아지면 초기화를 추천합니다

## 의존성 설치

- 하나하나 해봐요

In [1]:
!pip install torch torchvision torchaudio
!pip install librosa
!pip install pandas
!pip install tqdm
!pip install torchmetrics

- 라이브러리 입력

In [2]:
import librosa

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random

from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch
import torchmetrics
import os

In [3]:
import warnings
warnings.filterwarnings('ignore')

- 디바이스 선택
- 지금은 cpu만 될 것입니다

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

- 학습에 관한 설정파일
- sr = 음성이 초당 짤려지는 비율 이건 고정입니다
- n_mfcc = 음성의 특징을 가져올 개수
- batch_size = 한번에 학습하는 양
- n_epochs = 몇번 학습할 건가요?
- lr = 한번 학습할때 얼마나 조정을 하나요
- seed = 랜덤값 고정입니다. 아시잖아요 컴퓨터는 랜덤아니니까

In [5]:
class Config:
    SR = 32000
    N_MFCC = 52
    # Dataset
    ROOT_FOLDER = './'
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 5
    LR = 3e-4
    # Others
    SEED = 42
    
CONFIG = Config()

- 랜덤 고정

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED) # Seed 고정

- 데이터 파일 불러오기
- 학습인 train.csv에서 정보를 그대로 불러옵니다
- pandas라는 라이브러리의 데이터프레임을 사용합니다
- 데이터프레임은 csv와 같은 테이블입니다

In [7]:
df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

- 음성이잖아요? 그러니 음성에 맞는 특징을 가져옵니다

In [8]:
def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

- 함수를 실행해서 실제 값을 가져오는 겁니다.
- 특징을 뽑는 건데 좀 걸려요

In [9]:
train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

44350it [06:08, 120.29it/s]
11088it [01:30, 122.63it/s]


- 데이터에 대한 세트설정

In [10]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]
   


In [11]:
train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)

In [12]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

- 평가를 하는 것과 학습하는 것에 대한 세부사항

In [13]:
from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

- 학습을 할 모델

In [14]:
class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

- 학습시작
- 학습에서 train, val이 있습니다. train_loss는 학습데이터에 대한 오차, val_loss는 학습에 사용되지 않는 검증 데이터의 오차입니다.
- 같이 줄어들어야 좋습니다. val이 높아지면 큰일입니다
- auc 정확도입니다

In [15]:
model = MLP()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LR)

infer_model = train(model, optimizer, train_loader, val_loader, device)

100%|██████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 852.58it/s]


Epoch [1], Train Loss : [0.38785] Val Loss : [0.16761] Val AUC : [0.98719]


100%|██████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 838.17it/s]


Epoch [2], Train Loss : [0.17428] Val Loss : [0.08412] Val AUC : [0.99617]


100%|█████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 1193.36it/s]


Epoch [3], Train Loss : [0.12251] Val Loss : [0.05593] Val AUC : [0.99807]


100%|██████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 631.87it/s]


Epoch [4], Train Loss : [0.09748] Val Loss : [0.04663] Val AUC : [0.99854]


100%|█████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 1067.10it/s]

Epoch [5], Train Loss : [0.08627] Val Loss : [0.03638] Val AUC : [0.99900]


- 모델 저장
- 이름은 알아서 해주세요

In [16]:
torch.save(infer_model.state_dict(),"./mlp_model_01.pth")

- 제출을 위한 데이터 부분
- 실제 제출이 현재는 의미가 없어서 평가를 보여드립니다
- 실행해주세요

In [17]:
val.to_csv('val.csv',index=False)

In [18]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [19]:
val2 = pd.read_csv('./val.csv')
val2_mfcc = get_mfcc_feature(val2, False)
val2_dataset = CustomDataset(val2_mfcc, None)
val2_loader = DataLoader(
    val2_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

11088it [01:35, 116.10it/s]


In [20]:
preds = inference(infer_model, val2_loader, device)

100%|█████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 1626.31it/s]


In [21]:
submit = pd.read_csv('./val.csv')
submit.iloc[:, 1:] = preds
submit.head()

,id,path,label
0,PUOXNOKJ,0.000177234,0.999806
1,GXOIPDJP,0.990675,0.011206
2,FOEQKPPR,0.997121,0.00270101
3,IYASAVDT,0.00224651,0.997751
4,VLWIXPTC,0.124349,0.885412


In [22]:
df=submit.rename(columns={'path':'fake','label':'real'})

- 정확도를 산출

In [23]:
# 예측값을 기준으로 'real'과 'fake'로 변환
binary_preds = []
for i in preds:
    if i[0] < i[1]:
        binary_preds.append('real')
    else:
        binary_preds.append('fake')
        

# test.csv에서 실제 정답 레이블을 가져옴
true_labels = val['label'].values

# 예측값과 실제값이 일치하는지 확인
accuracy = np.mean(binary_preds == true_labels)

# 일치율을 퍼센트 형식으로 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 98.93%


- 데이터의 일치를 보실 수 있습니다

In [24]:
val2['pd'] = binary_preds
val2

,id,path,label,pd
0,PUOXNOKJ,./train/PUOXNOKJ.ogg,real,real
1,GXOIPDJP,./train/GXOIPDJP.ogg,fake,fake
2,FOEQKPPR,./train/FOEQKPPR.ogg,fake,fake
3,IYASAVDT,./train/IYASAVDT.ogg,real,real
4,VLWIXPTC,./train/VLWIXPTC.ogg,real,real
...,...,...,...,...
11083,WQMWFZRS,./train/WQMWFZRS.ogg,fake,fake
11084,KYLYAJSQ,./train/KYLYAJSQ.ogg,fake,fake
11085,AEFBUARF,./train/AEFBUARF.ogg,real,real
11086,VDPZMHZX,./train/VDPZMHZX.ogg,fake,fake


- 제출시에 사용

In [25]:
test = pd.read_csv('./test.csv')
test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

50000it [14:03, 59.25it/s]


In [29]:
test_model = MLP()
test_model.load_state_dict(torch.load("./mlp_model_01.pth"))
test_model.to(device)
test_model.eval()
preds = inference(test_model, test_loader, device)

100%|█████████████████████████████████████████████████████████████████████████████████████| 521/521 [00:00<00:00, 1153.03it/s]


In [30]:
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,TEST_00000,0.969886,0.038206
1,TEST_00001,0.832707,0.202042
2,TEST_00002,0.902915,0.120648
3,TEST_00003,0.085946,0.914218
4,TEST_00004,0.002521,0.997545


In [31]:
submit.to_csv('ssut.csv',index=False)

- dataset에 데이터를 저장합니다

In [32]:
#처리값
np.save('./dataset/train_mfcc.npy',train_mfcc)
np.save('./dataset/val_mfcc.npy',val_mfcc)
np.save('./dataset/test_mfcc.npy',test_mfcc)
np.save('./dataset/train_label.npy',train_labels)
np.save('./dataset/val_label.npy',val_labels)